# kalign

In [2]:
! /data/docker_qiime2_share_D/bionformatics-tools/kalign/base_3.3.2/bin/kalign -h


Kalign (3.3.2)

Copyright (C) 2006,2019,2020,2021 Timo Lassmann

This program comes with ABSOLUTELY NO WARRANTY; for details type:
`kalign -showw'.
This is free software, and you are welcome to redistribute it
under certain conditions; consult the COPYING file for details.

Please cite:
  Lassmann, Timo.
  "Kalign 3: multiple sequence alignment of large data sets."
  Bioinformatics (2019) 
  https://doi.org/10.1093/bioinformatics/btz795


Usage: kalign  -i <seq file> -o <out aln> 

Options:

   --format           : Output format. [Fasta]
   --reformat         : Reformat existing alignment. [NA]
   --gpo              : Gap open penalty. [5.5]
   --gpe              : Gap extension penalty. [2.0]
   --tgpe             : Terminal gap extension penalty. [1.0]
   --version (-V/-v)  : Prints version. [NA]

Examples:

Passing sequences via stdin:

   cat input.fa | kalign -f fasta > out.afa

Combining multiple input files:

   kalign seqsA.fa seqsB.fa seqsC.fa -f fasta > combined.afa



```Shell
time parallel -j 36 /data/docker_qiime2_share_D/bionformatics-tools/kalign/base_3.3.2/bin/kalign \
-i /data/docker_qiime2_share_G/jingjie/H_influenzae/core_genome/orthoslc/S10_write_fasta/strict_core/{} \
-o /data/docker_qiime2_share_G/jingjie/H_influenzae/core_genome/kalign/{} \
::: `ls /data/docker_qiime2_share_G/jingjie/H_influenzae/core_genome/orthoslc/S10_write_fasta/strict_core` > /data/docker_qiime2_share_G/jingjie/H_influenzae/core_genome/kaligned_log.txt 2>&1

```

# core genome construction

In [20]:
with open('/data/docker_qiime2_share_G/jingjie/H_influenzae/core_genome/orthoslc/Step1_op.txt', 'r') as f_read:
    rls = f_read.readlines()
    
f_read.close()

In [21]:
d = {}
for x in rls:
    l = x.split('\t')
    d[l[0]] = l[1]

In [22]:
from Bio.SeqRecord import SeqRecord
from Bio import SeqIO
from Bio.Seq import Seq

from multiprocessing import Process, Manager
import time
import os

import numpy as np

d

Seq('ATC') + Seq('AAA')

In [23]:
def concat(seq_rec_lst, shared_dict_, trans_d):
    for a_rec in seq_rec_lst:
        the_seq = str(shared_dict_[trans_d[a_rec.id[0: 5]]].seq)
        
        the_seq = the_seq + str(a_rec.seq)
        
        shared_dict_[trans_d[a_rec.id[0: 5]]] = SeqRecord(id = trans_d[a_rec.id[0: 5]], 
                                                             name = trans_d[a_rec.id[0: 5]],
                                                             description = "",
                                                             dbxrefs=[],
                                                             seq = Seq(the_seq)
                                                            )
#         print(shared_dict_[trans_d[a_rec.id[0: 5]]].seq)

def mission_spliter(a_lst, proc_num):
    n = len(a_lst)
    m = n // proc_num
    r = n % proc_num
    sublists = []
    start = 0
    for i in range(proc_num):
        end = start + m
        if i < r:
            end += 1
        sublists.append(a_lst[start:end])
        start = end
    return sublists

In [24]:
os.getcwd()

'/data/docker_qiime2_share_G/jingjie/H_influenzae/core_genome'

for x in ms:
    print(x.id[0: 5])

In [26]:
print(time.asctime())

proc_num = 36

if __name__ == "__main__":
    ini_seq = SeqIO.to_dict(SeqIO.parse("kalign/10052-11151.fasta", 
                                    "fasta")
                       )
    
    manager = Manager()
    shared_dict = manager.dict()
    
    for x in ini_seq.keys():
        shared_dict[d[x[0: 5]]] = SeqRecord(id = d[x[0: 5]], 
                                         name = d[x[0: 5]],
                                         description = "",
                                         dbxrefs=[],
                                         seq = Seq("")
                                        )
    
    
    for cls in os.listdir('kalign/'):
        a_cls = SeqIO.to_dict(SeqIO.parse('kalign/' + cls, 
                                    "fasta")
                       )
        
        m_lst = list(a_cls.values())
        
        splitted_m_lst = mission_spliter(m_lst, proc_num)
        
        jobs = []

        for ms in splitted_m_lst:
            p = Process(target = concat,
                       args = (list(ms), shared_dict, d)
                       )
          
            jobs.append(p)

            p.start()

    #     p.join()

        for z in jobs:
            z.join()
            
    to_save = list(shared_dict.values())
    
    with open("core_genome.fasta", "w") as output_handle:
        SeqIO.write(to_save, output_handle, "fasta")
        
    
print(time.asctime())

Sat Jan 28 10:11:49 2023
Sat Jan 28 10:12:39 2023


# to phy

In [ ]:
y_fasta = SeqIO.to_dict(SeqIO.parse("core_genome.fasta", 
                                    "fasta")
                       )

to_phy = list(y_fasta.values())

with open("core_genome_H_I_1281.phy", "w") as output_handle:
        SeqIO.write(to_phy, output_handle,'phylip-relaxed')

```Shell

cd /data/docker_qiime2_share_G/jingjie/H_influenzae/core_genome/raxml

raxmlHPC-PTHREADS-SSE3 \
-m GTRCAT -f a -p 1000 -# 1000 -x 1000 -s /data/docker_qiime2_share_G/jingjie/H_influenzae/core_genome/core_genome_H_I_1281.phy -n core_genome_H_I_1281 -T 33 > /data/docker_qiime2_share_G/jingjie/H_influenzae/core_genome/raxml/RAxML_logg.txt 2&>1

```